#### **Download FinBERT model from Hugging Face**

In [1]:
import os

if os.path.exists(os.path.abspath('./finbert_model/pytorch_model.bin')) == False:
    os.system('git lfs install')
    os.system('git clone https://huggingface.co/ProsusAI/finbert finbert_model')

#### **Add path to access FinBERT library**

In [2]:
import sys
import os

finbert_path = os.path.abspath('./finbert')
if (finbert_path in sys.path) == False:
    sys.path.append(finbert_path)

#### **Download NLTK tokenizer**

In [3]:
import nltk
nltk.download('punkt')

c:\Workspace\80_Framework\anaconda\envs\FinBERT\lib\site-packages\scipy\__init__.py:149: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.3
  UserWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xenos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### **Import libraries**

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import datetime
from finbert.finbert import predict
from transformers import AutoModelForSequenceClassification

ImportError: cannot import name 'predict' from 'finbert.finbert' (c:\Workspace\20_stock_market_prediction\finbert\finbert\__init__.py)

: 

#### **Create data output directory**

In [ ]:
output_dir = './output'
if os.path.exists(output_dir) == False:
    os.mkdir(output_dir)

#### **Access to Yahoo Finance page**  
Open Yahoo finance page using chrome driver and get main page news contents.  
News contents is wrapped by `container` class, so first get elements using this class info.  
After that, find `a` tag to find link element, and get title and link.  
In last, data will save as csv file.  

#### **TODO**  
Crawl news info from each category. Currently, this crawl main page news only.

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")

driver = webdriver.Chrome(chrome_options)
driver.get('https://finance.yahoo.com/')
driver.set_page_load_timeout(30)

elements = driver.find_elements(By.CLASS_NAME, 'container')

news_titles = []
for element in elements:
    # content_card = element.find_elements(By.CLASS_NAME, 'content')
    title_part = element.find_elements(By.TAG_NAME, 'a')
    if len(title_part) == 0:
        continue

    title = title_part[0].get_attribute('title')
    link = title_part[0].get_attribute('href')
    news_titles.append((title, link))

output_path = os.path.abspath('./output/' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '.csv')
with open(output_path, '+w', encoding='utf-8') as f:
    f.write('\n'.join([index[0] for index in news_titles]))
    f.close()

print("Main page news loaded.")
print(f"{len(elements)} contents found.")

#### **Load each articles contents**

In [ ]:
articles_index = []

for target_article in news_titles:
    driver.get(target_article[1])
    article_content_area = driver.find_elements(By.CLASS_NAME, 'caas-body')
    if len(article_content_area) == 0:
        continue

    article_content = []
    contents_components = article_content_area[0].find_elements(By.TAG_NAME, 'p')

    for component in contents_components:
        content = component.text
        if len(content) != 0:
            article_content.append(content)
    
    articles_index.append((target_article[0], article_content))
    print("Article parsed : " + target_article[0])

#### Open NYTimes page

In [ ]:
import requests
import json

with open('./tmp/config.json') as json_file:
    config = json.load(json_file)

api_key = config['api_key']

def get_articles(search: str):
    params = {
        'q': search,
        'api-key': api_key
    }
    res = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params)
    return (res.status_code == 200, json.loads(res.content))

def get_article_body(url: str):
    res = requests.get('https://www.nytimes.com/2024/05/23/business/apple-iphone-settlement.html')
    return (res.status_code == 200, res.content.decode())

In [17]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(chrome_options)

def get_nytimes_article(url: str):
    driver.get(url)
    driver.set_page_load_timeout(30)

    elements = driver.find_elements(By.CLASS_NAME, 'css-at9mc1')
    print(elements)

    article_content = []
    for element in elements:
        content = element.text
        if len(content) != 0:
            article_content.append(content)
        
    return article_content

In [18]:
get_nytimes_article('https://www.nytimes.com/2024/05/23/business/apple-iphone-settlement.html')

[<selenium.webdriver.remote.webelement.WebElement (session="ed4204fcfeb1ba7149bd217d9ad9e5ca", element="f.4E3F3CAD2E945F2BC89F1668C4FBD445.d.DCE61D173FA1E9956DE756A3656086BD.e.80")>, <selenium.webdriver.remote.webelement.WebElement (session="ed4204fcfeb1ba7149bd217d9ad9e5ca", element="f.4E3F3CAD2E945F2BC89F1668C4FBD445.d.DCE61D173FA1E9956DE756A3656086BD.e.81")>, <selenium.webdriver.remote.webelement.WebElement (session="ed4204fcfeb1ba7149bd217d9ad9e5ca", element="f.4E3F3CAD2E945F2BC89F1668C4FBD445.d.DCE61D173FA1E9956DE756A3656086BD.e.82")>]


[]

#### Parse articles

In [ ]:
articles_index = []

for target_article in news_titles:
    driver.get(target_article[1])
    article_content_area = driver.find_elements(By.CLASS_NAME, 'caas-body')
    if len(article_content_area) == 0:
        continue

    article_content = []
    contents_components = article_content_area[0].find_elements(By.TAG_NAME, 'p')

    for component in contents_components:
        content = component.text
        if len(content) != 0:
            article_content.append(content)
    
    articles_index.append((target_article[0], article_content))
    print("Article parsed : " + target_article[0])

#### **Save loaded article contents as file**

In [ ]:
import os

article_base_path = os.path.abspath('./output/' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '/') + '/'
os.mkdir(article_base_path)

for index in range(0, len(articles_index)):
    article = articles_index[index]
    article_path = article_base_path + str(index) + '.txt'

    with open(article_path, 'w', encoding='utf-8') as f:
        f.write(article[0] + '\n\n')
        f.write('\n'.join([index for index in article[1]]))
        f.close()

#### **Predict positive/negative using FinBERT model**

In [ ]:
model_path = os.path.abspath('./finbert_model')
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3, cache_dir=None)

output_path = os.path.abspath('./output/' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '_output/') + '/'
os.mkdir(output_path)
for index in range(0, len(articles_index)):
    article = articles_index[index]
    output_filepath = output_path + str(index) + '.csv'

    print(article[1])
    predict('\n'.join(article[1]), model, write_to_csv=True, path=output_filepath)